<a href="https://colab.research.google.com/github/brendontj/tcc-lol-stats/blob/master/LOL_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder   

In [2]:
df = pd.read_csv('Result_1.csv')
print(df.shape)

(69300, 60)


#Utils functions



In [3]:
def get_dragon_names(arr):
  dragons = []
  for i in arr:
    if not pd.isna(i):
      for j in i[1:-1].split(','):
        if j not in dragons:
          dragons.append(j)
  return dragons

def number_of_dragons(arr):
  if arr == 0:
    return 0
  return len(arr[1:-1].split(','))

def move_columns(df, cols_to_move, new_index):
    """
    This method re-arranges the columns in a dataframe to place the desired columns at the desired index.
    ex Usage: df = move_columns(df, ['Rev'], 2)   
    :param df:
    :param cols_to_move: The names of the columns to move. They must be a list
    :param new_index: The 0-based location to place the columns.
    :return: Return a dataframe with the columns re-arranged
    """
    other = [c for c in df if c not in cols_to_move]
    start = other[0:new_index]
    end = other[new_index:]
    return df[start + cols_to_move + end]

#Model to predict winner at 05 minutes of game

###Transform dragons data

In [4]:
df_1 = df.copy()

df_1['red_team_dragons'] = df_1['red_team_dragons'].fillna(0)
df_1['blue_team_dragons'] = df_1['blue_team_dragons'].fillna(0)
print(df_1.shape)
df_1['len_red_team_dragons'] = df_1['red_team_dragons'].apply(lambda x: number_of_dragons(x))
df_1['len_blue_team_dragons'] = df_1['blue_team_dragons'].apply(lambda x: number_of_dragons(x))
print(df_1.shape)

(69300, 60)
(69300, 62)


##Filter just MD1, because MD2,MD3 and MD5 are not labeled
### TODO - resolver possivel problema na extracao dos jogos md2/md3/md5

In [5]:
df_f = df_1.loc[df_1['best_of'] == 1]
print(df_f.shape)

(44380, 62)


In [215]:
name_of_columns = [
                                     'game_external_ref',
                                     'patch_version',
                                     'blue_team_ref',
                                     'red_team_ref',
                                     'blue_team_total_gold',
                                     'blue_team_inhibitors',
                                     'blue_team_towers',
                                     'blue_team_barons',
                                     'blue_team_total_kills',	
                                     'red_team_total_gold',
                                     'red_team_inhibitors',	
                                     'red_team_towers',	
                                     'red_team_barons',	
                                     'red_team_total_kills',
                                     'game_timestamp',
                                     'game_number',
                                     'team_a_side',
                                     'team_b_side',
                                     'team_a_record_wins',                                  
                                     'team_b_record_wins',
                                     'team_a_record_losses',
                                     'team_b_record_losses',                                                                     
                                     'best_of',
                                     'red_team_dragons',
                                     'blue_team_dragons',
                                     'participant_id_player_1',
                                     'champion_name_player_1',
                                     'role_player_1',
                                     'level_player_1',
                                     'kills_player_1',
                                     'deaths_player_1',
                                     'assists_player_1',
                                     'total_gold_earned_player_1',
                                     'creep_score_player_1',
                                     'kill_participation_player_1',
                                     'champion_damage_share_player_1',
                                     'wards_placed_player_1',
                                     'wards_destroyed_player_1',
                                     'participant_id_player_2',
                                     'champion_name_player_2',
                                     'role_player_2',
                                     'level_player_2',
                                     'kills_player_2',
                                     'deaths_player_2',
                                     'assists_player_2',
                                     'total_gold_earned_player_2',
                                     'creep_score_player_2',
                                     'kill_participation_player_2',
                                     'champion_damage_share_player_2',
                                     'wards_placed_player_2',
                                     'wards_destroyed_player_2',
                                     'participant_id_player_3',
                                     'champion_name_player_3',
                                     'role_player_3',
                                     'level_player_3',
                                     'kills_player_3',
                                     'deaths_player_3',
                                     'assists_player_3',
                                     'total_gold_earned_player_3',
                                     'creep_score_player_3',
                                     'kill_participation_player_3',
                                     'champion_damage_share_player_3',
                                     'wards_placed_player_3',
                                     'wards_destroyed_player_3',
                                     'participant_id_player_4',
                                     'champion_name_player_4',
                                     'role_player_4',
                                     'level_player_4',
                                     'kills_player_4',
                                     'deaths_player_4',
                                     'assists_player_4',
                                     'total_gold_earned_player_4',
                                     'creep_score_player_4',
                                     'kill_participation_player_4',
                                     'champion_damage_share_player_4',
                                     'wards_placed_player_4',
                                     'wards_destroyed_player_4',
                                     'participant_id_player_5',
                                     'champion_name_player_5',
                                     'role_player_5',
                                     'level_player_5',
                                     'kills_player_5',
                                     'deaths_player_5',
                                     'assists_player_5',
                                     'total_gold_earned_player_5',
                                     'creep_score_player_5',
                                     'kill_participation_player_5',
                                     'champion_damage_share_player_5',
                                     'wards_placed_player_5',
                                     'wards_destroyed_player_5',
                                     'participant_id_player_6',
                                     'champion_name_player_6',
                                     'role_player_6',
                                     'level_player_6',
                                     'kills_player_6',
                                     'deaths_player_6',
                                     'assists_player_6',
                                     'total_gold_earned_player_6',
                                     'creep_score_player_6',
                                     'kill_participation_player_6',
                                     'champion_damage_share_player_6',
                                     'wards_placed_player_6',
                                     'wards_destroyed_player_6',
                                     'participant_id_player_7',
                                     'champion_name_player_7',
                                     'role_player_7',
                                     'level_player_7',
                                     'kills_player_7',
                                     'deaths_player_7',
                                     'assists_player_7',
                                     'total_gold_earned_player_7',
                                     'creep_score_player_7',
                                     'kill_participation_player_7',
                                     'champion_damage_share_player_7',
                                     'wards_placed_player_7',
                                     'wards_destroyed_player_7',                                
                                     'participant_id_player_8',
                                     'champion_name_player_8',
                                     'role_player_8',
                                     'level_player_8',
                                     'kills_player_8',
                                     'deaths_player_8',
                                     'assists_player_8',
                                     'total_gold_earned_player_8',
                                     'creep_score_player_8',
                                     'kill_participation_player_8',
                                     'champion_damage_share_player_8',
                                     'wards_placed_player_8',
                                     'wards_destroyed_player_8',                                     
                                     'participant_id_player_9',
                                     'champion_name_player_9',
                                     'role_player_9',
                                     'level_player_9',
                                     'kills_player_9',
                                     'deaths_player_9',
                                     'assists_player_9',
                                     'total_gold_earned_player_9',
                                     'creep_score_player_9',
                                     'kill_participation_player_9',
                                     'champion_damage_share_player_9',
                                     'wards_placed_player_9',
                                     'wards_destroyed_player_9',                                
                                     'participant_id_player_10',
                                     'champion_name_player_10',
                                     'role_player_10',
                                     'level_player_10',
                                     'kills_player_10',
                                     'deaths_player_10',
                                     'assists_player_10',
                                     'total_gold_earned_player_10',
                                     'creep_score_player_10',
                                     'kill_participation_player_10',
                                     'champion_damage_share_player_10',
                                     'wards_placed_player_10',
                                     'wards_destroyed_player_10',
                                     'game_winner' # Game winner; 0 if team_a win the game and 1 if team_b win the game
                                     ]
df_5_minutes = pd.DataFrame(columns=name_of_columns)

In [216]:
games_ref_list = df_f['game_external_ref'].unique()

for i in range(len(games_ref_list)):
    sub_df = df_f.loc[df_f['game_external_ref'] == games_ref_list[i]]
    
    game_timestamp_list = sub_df['game_timestamp'].unique()
    
    micro_df = sub_df.loc[sub_df['game_timestamp'] == game_timestamp_list[5]]
    
    if int(micro_df.iloc[[0]].team_a_game_wins.values[0]) == 1 and int(micro_df.iloc[[0]].team_b_game_wins.values[0]) == 0:
      game_winner = 0
    elif int(micro_df.iloc[[0]].team_a_game_wins.values[0]) == 0 and int(micro_df.iloc[[0]].team_b_game_wins.values[0]) == 1:
      game_winner = 1

    data = [(micro_df.iloc[[0]].game_external_ref).values[0],
            (micro_df.iloc[[0]].patch_version).values[0],
            (micro_df.iloc[[0]].blue_team_ref).values[0],
            (micro_df.iloc[[0]].red_team_ref).values[0],        
            (micro_df.iloc[[0]].blue_team_total_gold).values[0],
            (micro_df.iloc[[0]].blue_team_inhibitors).values[0],
            (micro_df.iloc[[0]].blue_team_towers).values[0],
            (micro_df.iloc[[0]].blue_team_barons).values[0],
            (micro_df.iloc[[0]].blue_team_total_kills).values[0],
            (micro_df.iloc[[0]].red_team_total_gold).values[0],
            (micro_df.iloc[[0]].red_team_inhibitors).values[0],
            (micro_df.iloc[[0]].red_team_towers).values[0],          
            (micro_df.iloc[[0]].red_team_barons).values[0],
            (micro_df.iloc[[0]].red_team_total_kills).values[0],
            (micro_df.iloc[[0]].game_timestamp).values[0],
            (micro_df.iloc[[0]].game_number).values[0],
            (micro_df.iloc[[0]].team_a_side).values[0],
            (micro_df.iloc[[0]].team_b_side).values[0],
            (micro_df.iloc[[0]].team_a_record_wins).values[0],            
            (micro_df.iloc[[0]].team_b_record_wins).values[0],
            (micro_df.iloc[[0]].team_a_record_losses).values[0],
            (micro_df.iloc[[0]].team_b_record_losses).values[0],
            (micro_df.iloc[[0]].best_of).values[0],
            (micro_df.iloc[[0]].len_red_team_dragons).values[0],
            (micro_df.iloc[[0]].len_blue_team_dragons).values[0],
            (micro_df.iloc[[0]].participant_id).values[0],
            (micro_df.iloc[[0]].champion_name).values[0],
            (micro_df.iloc[[0]].role).values[0],
            (micro_df.iloc[[0]].level).values[0],
            (micro_df.iloc[[0]].kills).values[0],
            (micro_df.iloc[[0]].deaths).values[0],
            (micro_df.iloc[[0]].assists).values[0],
            (micro_df.iloc[[0]].total_gold_earned).values[0],
            (micro_df.iloc[[0]].creep_score).values[0],
            (micro_df.iloc[[0]].kill_participation).values[0],
            (micro_df.iloc[[0]].champion_damage_share).values[0],
            (micro_df.iloc[[0]].wards_placed).values[0],
            (micro_df.iloc[[0]].wards_destroyed).values[0],
            (micro_df.iloc[[1]].participant_id).values[0],
            (micro_df.iloc[[1]].champion_name).values[0],
            (micro_df.iloc[[1]].role).values[0],
            (micro_df.iloc[[1]].level).values[0],
            (micro_df.iloc[[1]].kills).values[0],
            (micro_df.iloc[[1]].deaths).values[0],
            (micro_df.iloc[[1]].assists).values[0],
            (micro_df.iloc[[1]].total_gold_earned).values[0],
            (micro_df.iloc[[1]].creep_score).values[0],
            (micro_df.iloc[[1]].kill_participation).values[0],
            (micro_df.iloc[[1]].champion_damage_share).values[0],
            (micro_df.iloc[[1]].wards_placed).values[0],
            (micro_df.iloc[[1]].wards_destroyed).values[0],           
            (micro_df.iloc[[2]].participant_id).values[0],
            (micro_df.iloc[[2]].champion_name).values[0],
            (micro_df.iloc[[2]].role).values[0],
            (micro_df.iloc[[2]].level).values[0],
            (micro_df.iloc[[2]].kills).values[0],
            (micro_df.iloc[[2]].deaths).values[0],
            (micro_df.iloc[[2]].assists).values[0],
            (micro_df.iloc[[2]].total_gold_earned).values[0],
            (micro_df.iloc[[2]].creep_score).values[0],
            (micro_df.iloc[[2]].kill_participation).values[0],
            (micro_df.iloc[[2]].champion_damage_share).values[0],
            (micro_df.iloc[[2]].wards_placed).values[0],
            (micro_df.iloc[[2]].wards_destroyed).values[0],           
            (micro_df.iloc[[3]].participant_id).values[0],
            (micro_df.iloc[[3]].champion_name).values[0],
            (micro_df.iloc[[3]].role).values[0],
            (micro_df.iloc[[3]].level).values[0],
            (micro_df.iloc[[3]].kills).values[0],
            (micro_df.iloc[[3]].deaths).values[0],
            (micro_df.iloc[[3]].assists).values[0],
            (micro_df.iloc[[3]].total_gold_earned).values[0],
            (micro_df.iloc[[3]].creep_score).values[0],
            (micro_df.iloc[[3]].kill_participation).values[0],
            (micro_df.iloc[[3]].champion_damage_share).values[0],
            (micro_df.iloc[[3]].wards_placed).values[0],
            (micro_df.iloc[[3]].wards_destroyed).values[0],           
            (micro_df.iloc[[4]].participant_id).values[0],
            (micro_df.iloc[[4]].champion_name).values[0],
            (micro_df.iloc[[4]].role).values[0],
            (micro_df.iloc[[4]].level).values[0],
            (micro_df.iloc[[4]].kills).values[0],
            (micro_df.iloc[[4]].deaths).values[0],
            (micro_df.iloc[[4]].assists).values[0],
            (micro_df.iloc[[4]].total_gold_earned).values[0],
            (micro_df.iloc[[4]].creep_score).values[0],
            (micro_df.iloc[[4]].kill_participation).values[0],
            (micro_df.iloc[[4]].champion_damage_share).values[0],
            (micro_df.iloc[[4]].wards_placed).values[0],
            (micro_df.iloc[[4]].wards_destroyed).values[0],          
            (micro_df.iloc[[5]].participant_id).values[0],
            (micro_df.iloc[[5]].champion_name).values[0],
            (micro_df.iloc[[5]].role).values[0],
            (micro_df.iloc[[5]].level).values[0],
            (micro_df.iloc[[5]].kills).values[0],
            (micro_df.iloc[[5]].deaths).values[0],
            (micro_df.iloc[[5]].assists).values[0],
            (micro_df.iloc[[5]].total_gold_earned).values[0],
            (micro_df.iloc[[5]].creep_score).values[0],
            (micro_df.iloc[[5]].kill_participation).values[0],
            (micro_df.iloc[[5]].champion_damage_share).values[0],
            (micro_df.iloc[[5]].wards_placed).values[0],
            (micro_df.iloc[[5]].wards_destroyed).values[0],           
            (micro_df.iloc[[6]].participant_id).values[0],
            (micro_df.iloc[[6]].champion_name).values[0],
            (micro_df.iloc[[6]].role).values[0],
            (micro_df.iloc[[6]].level).values[0],
            (micro_df.iloc[[6]].kills).values[0],
            (micro_df.iloc[[6]].deaths).values[0],
            (micro_df.iloc[[6]].assists).values[0],
            (micro_df.iloc[[6]].total_gold_earned).values[0],
            (micro_df.iloc[[6]].creep_score).values[0],
            (micro_df.iloc[[6]].kill_participation).values[0],
            (micro_df.iloc[[6]].champion_damage_share).values[0],
            (micro_df.iloc[[6]].wards_placed).values[0],
            (micro_df.iloc[[6]].wards_destroyed).values[0],           
            (micro_df.iloc[[7]].participant_id).values[0],
            (micro_df.iloc[[7]].champion_name).values[0],
            (micro_df.iloc[[7]].role).values[0],
            (micro_df.iloc[[7]].level).values[0],
            (micro_df.iloc[[7]].kills).values[0],
            (micro_df.iloc[[7]].deaths).values[0],
            (micro_df.iloc[[7]].assists).values[0],
            (micro_df.iloc[[7]].total_gold_earned).values[0],
            (micro_df.iloc[[7]].creep_score).values[0],
            (micro_df.iloc[[7]].kill_participation).values[0],
            (micro_df.iloc[[7]].champion_damage_share).values[0],
            (micro_df.iloc[[7]].wards_placed).values[0],
            (micro_df.iloc[[7]].wards_destroyed).values[0],
            (micro_df.iloc[[8]].participant_id).values[0],
            (micro_df.iloc[[8]].champion_name).values[0],
            (micro_df.iloc[[8]].role).values[0],
            (micro_df.iloc[[8]].level).values[0],
            (micro_df.iloc[[8]].kills).values[0],
            (micro_df.iloc[[8]].deaths).values[0],
            (micro_df.iloc[[8]].assists).values[0],
            (micro_df.iloc[[8]].total_gold_earned).values[0],
            (micro_df.iloc[[8]].creep_score).values[0],
            (micro_df.iloc[[8]].kill_participation).values[0],
            (micro_df.iloc[[8]].champion_damage_share).values[0],
            (micro_df.iloc[[8]].wards_placed).values[0],
            (micro_df.iloc[[8]].wards_destroyed).values[0],
            (micro_df.iloc[[9]].participant_id).values[0],
            (micro_df.iloc[[9]].champion_name).values[0],
            (micro_df.iloc[[9]].role).values[0],
            (micro_df.iloc[[9]].level).values[0],
            (micro_df.iloc[[9]].kills).values[0],
            (micro_df.iloc[[9]].deaths).values[0],
            (micro_df.iloc[[9]].assists).values[0],
            (micro_df.iloc[[9]].total_gold_earned).values[0],
            (micro_df.iloc[[9]].creep_score).values[0],
            (micro_df.iloc[[9]].kill_participation).values[0],
            (micro_df.iloc[[9]].champion_damage_share).values[0],
            (micro_df.iloc[[9]].wards_placed).values[0],
            (micro_df.iloc[[9]].wards_destroyed).values[0],
            game_winner]
   
    df_5_minutes = df_5_minutes.append(pd.Series(data, index=name_of_columns), ignore_index=True)


In [217]:
df_5_minutes = pd.get_dummies(df_5_minutes, columns=["patch_version"], prefix="patch_version_")

p1 = df_5_minutes['champion_name_player_1'].unique()
p2 = df_5_minutes['champion_name_player_2'].unique()
p3 = df_5_minutes['champion_name_player_3'].unique()
p4 = df_5_minutes['champion_name_player_4'].unique()
p5 = df_5_minutes['champion_name_player_5'].unique()
p6 = df_5_minutes['champion_name_player_6'].unique()
p7 = df_5_minutes['champion_name_player_7'].unique()
p8 = df_5_minutes['champion_name_player_8'].unique()
p9 = df_5_minutes['champion_name_player_9'].unique()
p10 =df_5_minutes['champion_name_player_10'].unique()

hero_names = np.unique(np.concatenate([p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]))

new_vals = [0 for i in range(len(hero_names))]
df_5_minutes = df_5_minutes.reindex(columns=df_5_minutes.columns.tolist() + hero_names.tolist())   # add empty cols
df_5_minutes[hero_names] = new_vals  # multi-column assignment works for existing cols

def fill_champion(df_5_minutes):
    df_5_minutes[df_5_minutes.champion_name_player_1] = -1
    df_5_minutes[df_5_minutes.champion_name_player_2] = -1
    df_5_minutes[df_5_minutes.champion_name_player_3] = -1
    df_5_minutes[df_5_minutes.champion_name_player_4] = -1
    df_5_minutes[df_5_minutes.champion_name_player_5] = -1

    df_5_minutes[df_5_minutes.champion_name_player_6] = 1
    df_5_minutes[df_5_minutes.champion_name_player_7] = 1
    df_5_minutes[df_5_minutes.champion_name_player_8] = 1
    df_5_minutes[df_5_minutes.champion_name_player_9] = 1
    df_5_minutes[df_5_minutes.champion_name_player_10] = 1

    return df_5_minutes

df_5_minutes = df_5_minutes.apply(fill_champion, axis=1)

print(df_5_minutes.shape)
df_5_minutes.head()

(634, 313)


,game_external_ref,blue_team_ref,red_team_ref,blue_team_total_gold,blue_team_inhibitors,blue_team_towers,blue_team_barons,blue_team_total_kills,red_team_total_gold,red_team_inhibitors,red_team_towers,red_team_barons,red_team_total_kills,game_timestamp,game_number,team_a_side,team_b_side,team_a_record_wins,team_b_record_wins,team_a_record_losses,team_b_record_losses,best_of,red_team_dragons,blue_team_dragons,participant_id_player_1,champion_name_player_1,role_player_1,level_player_1,kills_player_1,deaths_player_1,assists_player_1,total_gold_earned_player_1,creep_score_player_1,kill_participation_player_1,champion_damage_share_player_1,wards_placed_player_1,wards_destroyed_player_1,participant_id_player_2,champion_name_player_2,role_player_2,...,Sett,Shen,Shyvana,Sion,Sivir,Skarner,Soraka,Swain,Sylas,Syndra,TahmKench,Taliyah,Taric,Thresh,Tristana,Trundle,Tryndamere,TwistedFate,Twitch,Udyr,Urgot,Varus,Vayne,Veigar,Velkoz,Viego,Viktor,Vladimir,Volibear,Xayah,Xerath,XinZhao,Yasuo,Yone,Yorick,Yuumi,Zed,Ziggs,Zilean,Zoe
0,104727755403883497,103877554248683116,103935523328473675,26669,0,0,0,2,32174,0,3,0,7,2020-08-25 16:25:00.000000,1,blue,red,2,4,4,2,1,2,0,1,Ornn,top,11,0,0,0,5289,135,0.000000,0.459797,7,0,2,Hecarim,jungle,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,-1
1,104727755403883501,103877675241047720,103935523328473675,43464,0,4,1,10,37895,0,1,0,5,2020-08-27 17:17:00.000000,1,blue,red,4,4,2,2,1,3,1,1,Shen,top,14,1,1,6,8810,181,0.700000,0.106712,7,0,2,Hecarim,jungle,...,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,104727755403949039,104710359142494900,103877554248683116,39617,0,0,0,3,43664,0,4,1,2,2020-08-27 16:22:00.000000,1,blue,red,2,2,4,4,1,3,1,1,Sett,top,14,1,1,2,8107,212,1.000000,0.237772,13,3,2,Lillia,jungle,...,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,104727755403949041,103877554248683116,103877675241047720,41607,0,2,1,9,39436,0,3,0,3,2020-08-26 19:35:00.000000,1,blue,red,2,4,4,2,1,2,2,1,Shen,top,15,1,1,8,8371,185,1.000000,0.099533,8,3,2,Sett,jungle,...,-1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,104727755403949043,104710359142494900,103877675241047720,32886,0,1,0,6,33504,0,3,0,1,2020-08-25 17:17:00.000000,1,blue,red,2,4,4,2,1,2,1,1,Sett,top,12,1,0,4,6279,136,0.833333,0.241645,9,4,2,Lillia,jungle,...,-1,0,0,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [218]:
del df_5_minutes['champion_name_player_1']
del df_5_minutes['champion_name_player_2']
del df_5_minutes['champion_name_player_3']
del df_5_minutes['champion_name_player_4']
del df_5_minutes['champion_name_player_5']
del df_5_minutes['champion_name_player_6']
del df_5_minutes['champion_name_player_7']
del df_5_minutes['champion_name_player_8']
del df_5_minutes['champion_name_player_9']
del df_5_minutes['champion_name_player_10']
del df_5_minutes['role_player_1']
del df_5_minutes['role_player_2']
del df_5_minutes['role_player_3']
del df_5_minutes['role_player_4']
del df_5_minutes['role_player_5']
del df_5_minutes['role_player_6']
del df_5_minutes['role_player_7']
del df_5_minutes['role_player_8']
del df_5_minutes['role_player_9']
del df_5_minutes['role_player_10']
del df_5_minutes['blue_team_ref']
del df_5_minutes['red_team_ref']
del df_5_minutes['game_external_ref']
del df_5_minutes['game_timestamp']
del df_5_minutes['team_a_side']
del df_5_minutes['team_b_side']

In [219]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.utils.validation import column_or_1d

In [220]:
df_5_minutes = move_columns(df_5_minutes, ['game_winner'], 0)
X = df_5_minutes.iloc[:,1:]
y = df_5_minutes.iloc[:,:1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=2)

y_train = column_or_1d(y_train, warn=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  import sys


In [221]:
scaler = preprocessing.StandardScaler(with_mean=False).fit(X_train)
X_scaled = scaler.transform(X_train)

clf = LogisticRegression(max_iter=500).fit(X_scaled, y_train)
X_test_scaled = scaler.transform(X_test)
print(clf.score(X_test_scaled, y_test))

0.8125


In [222]:
clf = RandomForestClassifier(max_depth=300)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.65625


In [223]:
clf = KNeighborsClassifier(n_neighbors=30)
clf.fit(X_scaled, y_train)
print(clf.score(X_test_scaled, y_test))

0.78125


In [224]:
clf = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto', n_components=None, )
clf.fit(X_scaled, y_train)
print(clf.score(X_test_scaled, y_test))

0.84375
